In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_excel("Appel d'offre.xlsx")
df

,Poste,Lieu,Experience,Entité,Délai,Nb_Consultant,Salaire,Compétence
0,DATA,"Quai Saint-Exupéry, 92100 Paris",1,Microsoft,16,1,2000,SCALA;HADOOP;PYTHON;AZURE;ELK
1,JAVA,"3 Boulevard Ney, 75018 Paris",5,Amazon,4,4,2700,JAVA;C++;PYTHON;AWS;SQL;JAVASCRIPT;HIBERNATE
2,RH,"8 Rue de Londres, 75009 Paris",3,Google,7,7,1500,OFFICE365;ELK;HADOOP
3,CLOUD,"6 Rue Ménars, 75002 Paris",0,Alliexpress,31,0,470,SPARK;HTML;DJANGO;FLASK
4,UNIX,"12 Avenue Pierre Mendès-France, 75013 Paris",10,Carrefour,7,11,200,JAVASCRIPT;LINUX


In [3]:
lang = df['Compétence'].str.get_dummies(sep=';')
df = df.drop('Compétence',axis=1)

In [4]:
companies = pd.read_csv('companies_ranking.csv')

mask = []
for i in range(len(df)):
    if df.iloc[i]['Entité'] in companies.Entité.values:
        mask.append(df.iloc[i]['Entité'])
        #pd.concat(df.iloc[i],df[df['Entité']==df.iloc[i]['Entité']])

companies = companies.loc[companies['Entité'].isin(mask)]
companies

,Valeur,Entité
0,99,Microsoft
19,80,Google


In [5]:
for i in range(len(df)):
    if df.iloc[i]['Entité'] in companies.Entité.values:
        df['Entité'][i] = int(companies.loc[companies['Entité']==df.iloc[i]['Entité']]['Valeur'].values)
    else :
        df['Entité'][i] = 0

C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Utilisateur\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [6]:
def con_Scoring(x):
    if (x > 10) : 
        x = 1
    elif (x > 5) :
        x = 0.66 
    elif (x > 0) :
        x = 0.33
    else :
        x = 0
    return x

df['Nb_Consultant'] = df['Nb_Consultant'].apply(lambda x : con_Scoring(x))
df['Nb_Consultant']

0    0.33
1    0.33
2    0.66
3    0.00
4    1.00
Name: Nb_Consultant, dtype: float64

In [7]:
def exp_Scoring(x):
    if (x > 10) : 
        x = 1
    elif (x > 5) :
        x = 0.75
    elif (x > 2) :
        x = 0.5 
    elif (x > 0) :
        x = 0.25
    else :
        x = 0
    return x

df['Experience'] = df['Experience'].apply(lambda x : exp_Scoring(x))
df['Experience']

0    0.25
1    0.50
2    0.50
3    0.00
4    0.75
Name: Experience, dtype: float64

In [8]:
poste = df[['Poste']]
df.drop('Poste',axis=1,inplace=True)

Lieu = df[['Lieu']]
df.drop('Lieu',axis=1,inplace=True)

In [9]:
normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df

df = poste.join(normalized_df)

In [10]:
comp_E = []
comp_NE = []


data_comp_E = ['PYTHON','R', 'SCALA','C++','HADOOP','SPARK','JAVASCRIPT','DOCKER']
data_comp_NE = ['HIVE','HBASE','MESOS','COUCHBASE', 'ELK','KUBERNETES','SQL']

java_comp_E = ['JAVA', 'J2EE', 'SPRING', 'SPRING BOOT', 'REST']
java_comp_NE = ['HIBERNATE', 'GROOVY', 'GRAILS', 'ANGULAR', 'IAAS', 'AWS', 'ELK']

devops_comp_E = ['IAC','PACKER', 'ANSIBLE', 'SALTSTACK', 'PUPPET', 'CHEF','DOCKER','RANCHEROS','PYTHON','BASH','SHELL']
devops_comp_NE = ['JENKINS','Gitlab-CI', 'Travis-CI', 'XL-Deploy','KUBERNETES','AWS']


In [11]:
def lang_scoring(i,e,ne):
    for x in lang:
        
            if x in e:
                lang.iloc[i][x] = lang.iloc[i][x]*2
            elif x in ne:
                lang.iloc[i][x] = lang.iloc[i][x]*1
            else :
                lang.iloc[i][x] = 0

In [12]:
for i in range(0,len(df)):
    if df.iloc[i]['Poste'] == 'DATA':
        lang_scoring(i,data_comp_E,data_comp_NE)
    elif df.iloc[i]['Poste'] == 'JAVA':
        lang_scoring(i,java_comp_E,java_comp_NE)
    elif df.iloc[i]['Poste'] == 'DEVOPS':
        lang_scoring(i,devops_comp_E,devops_comp_NE)
    else :
        lang_scoring(i,comp_E,comp_NE)

In [13]:
lang['lang_Score'] = lang.sum(axis=1)
lang

,AWS,AZURE,C++,DJANGO,ELK,FLASK,HADOOP,HIBERNATE,HTML,JAVA,JAVASCRIPT,LINUX,OFFICE365,PYTHON,SCALA,SPARK,SQL,lang_Score
0,0,0,0,0,1,0,2,0,0,0,0,0,0,2,2,0,0,7
1,1,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
Score = lang['lang_Score']
Score =(Score-Score.min())/(Score.max()-Score.min())

In [15]:
normalized_df = normalized_df.join(Score)
normalized_df

,Experience,Entité,Délai,Nb_Consultant,Salaire,lang_Score
0,0.333333,1,0.444444,0.33,0.720,1.000000
1,0.666667,0,0.000000,0.33,1.000,0.571429
2,0.666667,0.808081,0.111111,0.66,0.520,0.000000
3,0.000000,0,1.000000,0.00,0.108,0.000000
4,1.000000,0,0.111111,1.00,0.000,0.000000


In [16]:
import requests, json
import urllib.parse
from math import pi,sqrt,sin,cos,atan2

api_url = "https://api-adresse.data.gouv.fr/search/?q="

def get_add(x):
    adr = x
    r = requests.get(api_url + urllib.parse.quote(adr))
    data = json.loads(r.content)
    
    return data.get('features')[0].get("geometry").get('coordinates')

Lieu['coord'] = Lieu['Lieu'].apply(lambda x : get_add(x))

In [17]:
add_op = '29 Rue des Sablons, 75116 Paris'

add_op = get_add(add_op)
lat_op = add_op[1]
lon_op = add_op[0]

from math import sin, cos, acos, pi
 
def distanceGPS(longB, latB):
    
    degree_to_rad = float(pi / 180.0)
    
    d_lat = (latB - lat_op) * degree_to_rad
    d_long = (longB - lon_op) * degree_to_rad
    a = pow(sin(d_lat / 2), 2) + cos(lat_op * degree_to_rad) * cos(latB * degree_to_rad) * pow(sin(d_long / 2), 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    km = 6367 * c
    return km
 
Lieu['distance'] = Lieu['coord'].apply(lambda x : distanceGPS(x[0],x[1]))

In [18]:
def dist_score(x):
    if (x > 30) : 
        x = 0.33
    elif (x > 15) :
        x = 0.66 
    elif (x <= 15) :
        x = 1
    else :
        x = 0
    return x


Lieu['dist_score'] = Lieu['distance'].apply(lambda x : dist_score(x))
dist_score = Lieu[['dist_score']]
normalized_df = normalized_df.join(dist_score)

In [19]:
def En_coeff(x):
    x = x*0.2
    return x
def Con_coeff(x):
    x= x*2
    return x
def lang_coeff(x):
    x = x*1.5
    return x

def dist_coeff(x):
    x = x*0.5
    return x
    

def get_score(x):
    x['Entité'] = x['Entité'].apply(lambda x : En_coeff(x))
    x['Nb_Consultant'] =  x['Nb_Consultant'].apply(lambda x : Con_coeff(x))
    x['lang_Score'] =  x['lang_Score'].apply(lambda x : lang_coeff(x))
    x['dist_score'] =  x['dist_score'].apply(lambda x : dist_coeff(x))


In [20]:
get_score(normalized_df)
normalized_df

,Experience,Entité,Délai,Nb_Consultant,Salaire,lang_Score,dist_score
0,0.333333,0.200000,0.444444,0.66,0.720,1.500000,0.5
1,0.666667,0.000000,0.000000,0.66,1.000,0.857143,0.5
2,0.666667,0.161616,0.111111,1.32,0.520,0.000000,0.5
3,0.000000,0.000000,1.000000,0.00,0.108,0.000000,0.5
4,1.000000,0.000000,0.111111,2.00,0.000,0.000000,0.5


In [21]:
normalized_df['Score'] = normalized_df.sum(axis=1)
normalized_df.sort_values(by='Score',ascending=False)

,Experience,Entité,Délai,Nb_Consultant,Salaire,lang_Score,dist_score,Score
0,0.333333,0.200000,0.444444,0.66,0.720,1.500000,0.5,4.357778
1,0.666667,0.000000,0.000000,0.66,1.000,0.857143,0.5,3.683810
4,1.000000,0.000000,0.111111,2.00,0.000,0.000000,0.5,3.611111
2,0.666667,0.161616,0.111111,1.32,0.520,0.000000,0.5,3.279394
3,0.000000,0.000000,1.000000,0.00,0.108,0.000000,0.5,1.608000
